In [16]:
import numpy as np
import torch

from torch import nn
from torch.utils.data import TensorDataset, DataLoader, RandomSampler

import torchvision
import matplotlib.pyplot as plt

from os import listdir
from PIL import Image

In [2]:
IMAGE_PATH = '../data/images'
LABEL_PATH = '../data/annotations'

In [3]:
def get_class_map():
    ret = {}

    i = 0
    for fname in listdir(LABEL_PATH):
        img_class, _ = fname.split('.')
        ret[img_class] = i
        i += 1

    return ret

In [18]:
def get_class_map_totext():
    ret = {}

    i = 0
    for fname in listdir(LABEL_PATH):
        img_class, _ = fname.split('.')
        ret[i] = img_class
        i += 1

    return ret

In [39]:
def get_dataloader(bs=64):
    data = []
    X = []
    y = []

    # mapping from class names to integers
    class_map = get_class_map()

    # loop through all the annotations
    for fname in listdir(LABEL_PATH):
        img_class, _ = fname.split('.')
    
        # open the annotation
        with open(f'{LABEL_PATH}/{fname}', 'r') as fh:

            # get image ids from annotation file
            img_ids = fh.read().splitlines()

            # gather the images with labels
            for img_id in img_ids:
                img_path = f'{IMAGE_PATH}/im{img_id}.jpg'
                img = Image.open(img_path)
                #img.show(img_id)
                img_data = np.asarray(img)
                
                # skip black-and-white images
                if not len(img_data.shape) == 3:
                    continue

                img_data = img_data.flatten().astype(np.float32)

                data.append([img_data, class_map[img_class]])

    return DataLoader(data, batch_size=bs, shuffle=True)

In [42]:
class TwoLayerModel(nn.Module):
    def __init__(self, n_input, n_hidden1, n_hidden2, n_classes):
        super().__init__()

        self.input_layer = nn.Linear(n_input, n_hidden1)
        self.hidden1 = nn.Linear(n_hidden1, n_hidden2)
        self.hidden2 = nn.Linear(n_hidden2, n_classes)
        self.relu = nn.ReLU()
        #self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.input_layer(x)
        x = self.relu(x)
        x = self.hidden1(x)
        x = self.relu(x)
        x = self.hidden2(x)
        #x = self.softmax(x)

        return x

In [36]:
def train(dataloader, model, optimizer, criterion, device, n_epochs=50, losses=[]):

    model.train()

    for epoch in range(n_epochs):
        
        for i, batch in enumerate(dataloader):
            X, y = batch
            X = X.to(device)
            y = y.to(device)

            optimizer.zero_grad()
            y_pred = model(X)
            loss = criterion(y_pred, y)
            loss.backward()
            optimizer.step
            
            losses.append(loss)

        print(f'Epoch: {epoch}, loss: {loss}')

In [34]:
# NOT WORKING YET
# Visualize some samples 
# Taken from transfer learning tutorial https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html
def imshow(inp, title=None):
    """Imshow for Tensor."""
    #inp = inp.numpy().transpose((1, 2, 0))
    #mean = np.array([0.485, 0.456, 0.406])
    #std = np.array([0.229, 0.224, 0.225])
    #inp = std * inp + mean
    #inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated

ntoshow = 1
dataloaderforvisu = get_dataloader(ntoshow)
#class_map_forvisu = get_class_map()
class_names = get_class_map_totext()
    
# Get a batch of training data
inputs, classes = next(iter(dataloaderforvisu))

inputs = inputs.reshape((ntoshow, 128, 128, 3))
print("Inputs shape")
print(inputs.shape)
print('Inputs:')
print(inputs)

print("Array data type")
print(inputs.dtype)

print('Classes:')
print(classes)
print("Classes shape")
print(classes.shape)


# Make a grid from batch
out = torchvision.utils.make_grid(inputs)


imshow(out)
#imshow(out, title=[class_names[x] for x in classes])

KeyboardInterrupt: 

In [44]:
use_cuda = True

device = torch.device('cuda') if use_cuda else torch.device('cpu')

lr = 0.05
n_epochs = 5
bs = 256

n_classes = len(get_class_map().keys())

model = TwoLayerModel(128*128*3, 1024, 512, n_classes).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=lr)

In [45]:
dataloader = get_dataloader(bs)

In [46]:
train(dataloader, model, optimizer, criterion, device, n_epochs)

Epoch: 0, loss: 29.312047958374023
Epoch: 1, loss: 29.278907775878906
Epoch: 2, loss: 29.68074607849121
Epoch: 3, loss: 30.50567054748535
Epoch: 4, loss: 30.045175552368164
